In [1]:
import json
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import os
import random
from transformers import AutoTokenizer

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='0'
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

In [3]:
with open('korquad_sim_sep.json', 'r', encoding='utf8') as fp:
    pairs = json.load(fp)

In [4]:
#sbert_iq = SentenceTransformer('../model_results/sbert_iq_all')
ours_ad = SentenceTransformer('../model_results_ft/ours_ad_exc_neu')
ours_ad_st = SentenceTransformer('../model_results_ft/ours_ad_st_exc_neu')
ours_curr = SentenceTransformer('../model_results_ft/ours_curr_exc_neu')

#sbert_iq.to('cuda')
ours_ad.to('cuda')
ours_ad_st.to('cuda')
ours_curr.to('cuda')
print('')

In [5]:
# pairs =[]
# for cnt, i in enumerate(test_data['data']):
#     for p in i['paragraphs']:
# #      `   if len(tokenizer.tokenize(p['doc']))<510:
#         for qs in p['qas']:
#             rn1 = random.randint(0,len(test_data))
#             while rn1 == cnt:
#                 rn1 = random.randint(0,len(test_data)-1)
#             rn2 = random.randint(0, len(test_data['data'][rn1]['paragraphs'])-1)
#             pairs.append({
#                 'sent': qs['question'],
#                 'doc': p['context'],
#                 'neg_doc':test_data['data'][rn1]['paragraphs'][rn2]['context']
#             })


In [6]:
# # less than 510 tokens
# pairs =[]
# for cnt, i in enumerate(test_data['data']):
#     for p in i['paragraphs']:
#         if len(tokenizer.tokenize(p['context']))<500:
#             for qs in p['qas']:
#                 rn1 = random.randint(0,len(test_data))
#                 while rn1 == cnt:
#                     rn1 = random.randint(0,len(test_data)-1)
#                 rn2 = random.randint(0, len(test_data['data'][rn1]['paragraphs'])-1)
                
#                 while len(tokenizer.tokenize(test_data['data'][rn1]['paragraphs'][rn2]['context']))>510:
#                     rn1 = random.randint(0,len(test_data))
#                     while rn1 == cnt:
#                         rn1 = random.randint(0,len(test_data)-1)
#                     rn2 = random.randint(0, len(test_data['data'][rn1]['paragraphs'])-1)

#                 pairs.append({
#                     'sent': qs['question'],
#                     'doc': p['context'],
#                     'neg_doc':test_data['data'][rn1]['paragraphs'][rn2]['context']
#                 })


In [7]:
psg = [i['doc'] for i in pairs]
sents = [i['sent'] for i in pairs]
psg_st = ['[PSG] ' +i['doc'] for i in pairs]
sents_st = ['[SENT] '+i['sent'] for i in pairs]
neg_psg = [i['neg_doc'] for i in pairs]
neg_psg_st = ['[PSG] '+i['neg_doc'] for i in pairs]

In [8]:
len(neg_psg_st)

5774

In [9]:
def psg_sent_vec(psg, sents, neg_psg, model):
    vec1 = model.encode(psg)
    vec2 = model.encode(sents)
    vec3 = model.encode(neg_psg)
    return vec1, vec2, vec3
def knn(index_vec, test_vec):
    knn_model = NearestNeighbors(n_neighbors=3,
                     metric='cosine',
                     algorithm='brute',
                     n_jobs=-1)
    knn_model.fit(index_vec)
    res = knn_model.kneighbors(test_vec, 3, return_distance=True)
    
    return res

def cos_sim(psg, sent, neg_psg, model):
    pvec,svec, npvec = psg_sent_vec(psg, sent,neg_psg, model)
    cos_sim1 = cosine_similarity([pvec, svec])[0][1]
    cos_sim2 = cosine_similarity([npvec, svec])[0][1]
    return cos_sim1, cos_sim2
    #print(m_name,': ' ,cos_sim,)
    
def print_sim(text,pos, neg, model, model_name):
    cn1,cn2 = 0,0
    for t,p,n in zip(text,pos,neg):
        c1, c2 = cos_sim(p,t,n, model)
        cn1 += c1
        cn2 += c2
    print(model_name+'_pos: ', cn1/ len(text))
    print(model_name+'_neg: ', cn2/ len(text))
        

In [10]:
# print_sim(sents, psg, neg_psg, sbert_iq, 'sb_iq')

In [11]:
print_sim(sents, psg, neg_psg, ours_ad, 'ours_ad')

ours_ad_pos:  0.6833600657113003
ours_ad_neg:  0.17224350360782342


In [12]:
print_sim(sents_st, psg_st, neg_psg_st, ours_ad_st, 'ours_ad_st')

ours_ad_st_pos:  0.6761248168952642
ours_ad_st_neg:  0.16827534865340762


In [13]:
print_sim(sents_st, psg_st, neg_psg_st, ours_curr, 'ours_curr')

ours_curr_pos:  0.6045832274830698
ours_curr_neg:  0.1382223948594675
